# US Accidents Exploration with Python
## A look into the number of accidents in the US from 2016 to 2019


This project is part of the Udacity Data Scientist for Enterprise nanodegree. The goal of the project is to find a dataset, come up with three questions to answer from it using Python, then writing a post on Medium to explain the findings.

The project has to follow the CRISP-DM industry standards for data science:

* Business Understanding 
* Data Understanding
* Prepare Data|
* Model Data
* Results
* Deploy

The questions I will tackle using this methodology are:

* What states have the highest number of accidents?
* What's the trend of the number of accidents and is there any seasonality?
* What impacts the number of accidents in the US?

The dataset used can be found on [Kaggle](https://www.kaggle.com/sobhanmoosavi/us-accidents/data) and the finding on [Medium](https://medium.com/r/?url=https%3A%2F%2Fwww.kaggle.com%2Fsobhanmoosavi%2Fus-accidents%2Fdata). 


In [1]:
# Importing libraries
import pandas as pd
import numpy as np
from pandas.tseries.offsets import MonthBegin
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import pandas as pd
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor

# Magic Keywords
%matplotlib inline

# Setting Theme
PlotsFont = 'Helvetica'

In [ ]:
# Reading data
AccidentData = pd.read_csv('US_Accidents_Dec19.csv')

## Data Exploration

In [ ]:
# Data Description
print('Data Description and Shape: ')
print(AccidentData.shape)
print(AccidentData.describe().T)

In [ ]:
# Checking Missing Values
plt.figure(figsize=(14, 4), dpi=80, facecolor='w', edgecolor='k')
plt.xticks(rotation=90, fontname = PlotsFont)
plt.yticks(fontname = PlotsFont)
plt.title('Count of Missing Values by Feature', fontname = PlotsFont, size = 12)
sns.barplot(x = AccidentData.isna().sum().index, y = AccidentData.isna().sum().values);

## Question 1: What states have the highest number of accidents?

- How do the states rank in terms of accidents?
- Does that have an impact on other variables?

In [ ]:
# Number of accidents per state in the dataset
plt.figure(num=None, figsize=(14, 4), dpi=80, facecolor='w', edgecolor='k')
plt.xticks(rotation=90, fontname = PlotsFont)
plt.yticks(fontname = PlotsFont)
plt.title('Number of Accident by State', fontname = PlotsFont, size = 12)
sns.barplot(x = AccidentData['State'].value_counts().index, y = AccidentData['State'].value_counts().values);

In [ ]:
# Getting states and values
States = list(AccidentData['State'].value_counts().index)
Values = list(AccidentData['State'].value_counts().values)

# Creating dataframe with the count of accidents per state.
MapData = pd.DataFrame({'code': States, 'accidents':Values})

# Plotting a map using plotly
fig = go.Figure(data=go.Choropleth(
    locations=MapData['code'],
    z=MapData['accidents'].astype(float),
    locationmode='USA-states',
    colorscale='Reds',
    marker_line_color='white', 
    colorbar_title="Car Accidents", ))

fig.update_layout(
    title_text='2016 - 2019 US Car Accidents',
    geo = dict(scope='usa', projection=go.layout.geo.Projection(type = 'albers usa'), showlakes=True, 
                lakecolor='rgb(255, 255, 255)'))

fig.show()

In [ ]:
# Calculating the number of accidents per weather condition and filtering on count > 500
AccidentsWeather = AccidentData['Weather_Condition'].value_counts()
AccidentsWeather = AccidentsWeather[AccidentsWeather.values > 500]

# Plotting bar chart
plt.figure(num=None, figsize=(14, 4), dpi=80, facecolor='w', edgecolor='k')
plt.xticks(rotation=90, fontname = PlotsFont)
plt.yticks(fontname = PlotsFont)
plt.title('Number of Accident by State', fontname = PlotsFont, size = 12)
sns.barplot(x = AccidentsWeather.index, y = AccidentsWeather.values, palette = 'Reds_r');

## What's the trend of the number of accidents and is there any seasonality?

- Is the number of accidents increasing or decreasing?
- Is there any seasonality in the number of accidents?

In [ ]:
# Checking the number of nulls in the start time column
print(f'Count of nulls in start time {str(AccidentData.Start_Time.isna().sum())} \n')

# Creating a column to hold the value per month. Calling it period
AccidentData['Period']  = pd.to_datetime(AccidentData.Start_Time).dt.to_period('M').dt.to_timestamp()

# The values of 2016 seem to be increasing which can show that the data collection wasn't done 
# compeltely in the beggining. 
# the data for 2020 also seems to be incomplete.
# Delete 2015, 2016 and 2020 for the data

AccidentData = AccidentData.loc[~AccidentData['Start_Time'].str.startswith('2015')]
AccidentData = AccidentData.loc[~AccidentData['Start_Time'].str.startswith('2016')]
AccidentData = AccidentData.loc[~AccidentData['Start_Time'].str.startswith('2020')]


In [ ]:
# Number of accidents per state in the dataset
fig = plt.figure(num=None, figsize=(14, 4), dpi=80, facecolor='w', edgecolor='k')
plt.xticks(rotation=90, fontname = PlotsFont)
plt.yticks(fontname = PlotsFont)
plt.title('Number of Accidents in the US over time', fontname = PlotsFont, size = 12)
sns.lineplot(data = AccidentData[['Period', 'ID']].groupby('Period').agg(['count']),markers=True, 
             dashes=False, palette = 'Reds', legend = False);


In [ ]:
GroupedData = pd.DataFrame({'count' : AccidentData.groupby( [ "Period", "State"] ).size()}).reset_index()
Top10Countries = AccidentData['State'].value_counts()[:10].index.values

# Top countries values
GroupedData = GroupedData[GroupedData['State'].isin(Top10Countries)]

HeatMapData = GroupedData.pivot("State", "Period")

plt.figure(num=None, figsize=(18, 4), dpi=80, facecolor='w', edgecolor='k')
plt.title('Evolution Top 10 States', fontname = PlotsFont, size = 12)
sns.heatmap(HeatMapData, cmap="YlGnBu", xticklabels = False);

In [ ]:
# Timeline data
TimelineData = pd.DataFrame({'count' : AccidentData.groupby(["Period"]).size()}).reset_index()

# Set Index of timeline
TimelineData['Period'] = pd.to_datetime(TimelineData['Period'])
TimelineData = TimelineData.set_index('Period')

# Setting the frequency for the decompose function
TimelineData.index.freq = 'MS'

# Decomposition
Decomposition = sm.tsa.seasonal_decompose(TimelineData)

# Plotting
fig, axes = plt.subplots(4, 1, sharex=True, figsize=(16, 10))
plt.suptitle('Timeline Decomposition')
Decomposition.observed.plot(ax=axes[0], legend=False, color = 'lightcoral')
axes[0].set_ylabel('Observed')
Decomposition.trend.plot(ax=axes[1], legend=False, color='lightcoral')
axes[1].set_ylabel('Trend')
Decomposition.seasonal.plot(ax=axes[2], legend=False, color = 'lightcoral')
axes[2].set_ylabel('Seasonal')
Decomposition.resid.plot(ax=axes[3], legend=False, color='lightcoral');
axes[3].set_ylabel('Residual');

## Question 3: What are the factors that have the biggest impact on the number of accidents?

- Can we find a variable that increases the number of accidents?
- Out of all the junction and sign types, what's the most effective to reduce the number of accidents?

In [ ]:
# Select columns
SelectedColumns = ['Amenity', 'Bump', 'Crossing',
       'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station',
       'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop',
       'Sunrise_Sunset']

# Filter Data
SelectedData = AccidentData[SelectedColumns]

# Dropping null values in Sunrise and Sunset. only 59 out of 3mil
SelectedData = SelectedData[~SelectedData.Sunrise_Sunset.isna()]

# Grouping data with counts
SelectedCount = pd.DataFrame({'Count' : AccidentData.groupby(SelectedColumns).size()}).reset_index()

# Splitting column types
SingleValue = [Column for Column in SelectedColumns if len(SelectedCount[Column].unique()) == 1]
BinaryColumns = [Column for Column in SelectedColumns if len(SelectedCount[Column].unique()) == 2]
CategoricalColumns = [Column for Column in SelectedColumns if len(SelectedCount[Column].unique()) > 2]

# Drop columns with single value
SelectedCount = SelectedCount.drop(SingleValue, axis = 1)

# Map binary columns
for Binary in BinaryColumns:
    UniqueValues = SelectedCount[Binary].unique()
    MappingDict = {oldval:newval for newval, oldval in enumerate(UniqueValues)}
    SelectedCount[Binary] = SelectedCount[Binary].map(MappingDict)

# Transforming categorical to binary
for Categorical in CategoricalColumns:
    SelectedCount = pd.concat([SelectedCount, pd.get_dummies(SelectedCount[Categorical])], axis=1)
    SelectedCount = SelectedCount.drop(Categorical, axis = 1)

In [ ]:
# Splitting X and Y
y = SelectedCount['Count']
X = SelectedCount.drop('Count', axis = 1)

# Create train and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state=42)

#Instantiate
lm_model = LinearRegression(normalize=True) 

# Fit the model
lm_model.fit(X_train, y_train) 

# Predict
y_test_preds = lm_model.predict(X_test)
y_train_preds = lm_model.predict(X_train)

# Calculate the R2 score for train and test
r2_scores_test = r2_score(y_test, y_test_preds)
r2_scores_train = r2_score(y_train, y_train_preds)

print(r2_scores_train)

In [ ]:
def coef_weights(coefficients, X_train):
    # Using function from nanodegree to sort coefficients
    coefs_df = pd.DataFrame()
    coefs_df['Variable'] = X_train.columns
    coefs_df['coefs'] = lm_model.coef_
    coefs_df['abs_coefs'] = np.abs(lm_model.coef_)
    coefs_df = coefs_df.sort_values('abs_coefs', ascending=False)
    return coefs_df

#Use the function
coef_df = coef_weights(lm_model.coef_, X_train)

#A quick look at the top results
coef_df.head(10)

# Barplot
plt.figure(num=None, figsize=(18, 10), dpi=80, facecolor='w', edgecolor='k')
sns.barplot(y = coef_df['Variable'], x = coef_df['coefs'], 
           palette = ['skyblue' if coef > 0 else 'lightcoral' for coef in coef_df['coefs']]);